In [112]:
import polars as pl
from polars import col as c
import networkx as nx

from config import settings
import json
import os
from datetime import datetime, UTC
import datetime as dt

from utility.polars_operation import generate_uuid_col
from utility.parser_utility import (
    add_table_to_changes_schema,
    generate_connectivity_table,
)

from twindigrid_changes.schema import ChangesSchema
from twindigrid_sql.schema.enum import TerminalSide
from twindigrid_sql.entries.source import SCADA

from twindigrid_sql.entries.equipment_class import (
    TRANSFORMER,
    BRANCH,
    SWITCH,
    INDIRECT_FEEDER,
    BUSBAR_SECTION,
    ENERGY_CONSUMER,
)

# Useless outside jupiternotebook because in settings.py a line that changes the directory to src for ipynb
os.chdir(os.getcwd().replace("/src", ""))
# os.getcwd()

# Import data from matlab

In [113]:
file_names: dict[str, str] = json.load(open(settings.INPUT_FILE_NAMES))

In [114]:
parameter_distflow: pl.DataFrame = pl.read_csv(file_names["Distflow_parameter"])
nodedata_distflow: pl.DataFrame = pl.read_csv(file_names["Distflow_node_data"])
powerdata_distflow: pl.DataFrame = pl.read_csv(file_names["Distflow_Power_data"])
linedata_distflow: pl.DataFrame = pl.read_csv(file_names["Distflow_Line_data"])
result_distflow: pl.DataFrame = pl.read_csv(file_names["Distflow_result"])
# nodedata_distflow = nodedata_distflow.with_columns(c("Snom").cast(pl.Int8))
# # To have base value (need lenght of line), not from matlab !
# line_data_from_input_file: pl.DataFrame = pl.read_excel(
#     file_names["Line_Data_From_Input_File"]
# )

# Add node number to power data
powerdata_distflow = powerdata_distflow.with_row_index(
    "node_number", offset=1
)  # offset=1 because slack bus is 0 and no power on it
powerdata_distflow = powerdata_distflow.with_columns(c("node_number").cast(pl.Int64))
# Create a topology dataframe with basic topology information

df_topology = nodedata_distflow.select(
    c("index").alias("node_number"),
    c("indexLines_1").alias("index_branch_1"),
    c("indexLines_2").alias("index_branch_2"),
    c("indexLines_3").alias("index_branch_3"),
    c("Vnom"),
)

# Add the power data to the topology dataframe with node as key
df_topology = df_topology.join(
    powerdata_distflow, on="node_number", how="full", coalesce=True
)
linedata_distflow.head(2)

node_from,node_to,line_number,r_pu,x_pu,b_pu,i_pu,connection_type
i64,i64,i64,f64,f64,f64,f64,i64
1,0,1,0.19375,0.112969,7.9168e-8,0.024942,2
2,0,2,2.046875,1.14625,8.9632e-7,0.026327,2


# Set missing value for equipment

In [115]:
### Set missing value for equipment
# Fake value for the length of the branch
base_length = 1
# Fake value for the switch state
switch_state = False
switch_type = "locked_switch"
switch_command = "unknown"

## Branch

In [116]:
# branch :pl.DataFrame =
from polars import Null


default_install_date: datetime = datetime(*settings.DEFAULT_INSTALL_DATE, tzinfo=UTC)
heartbeat = datetime.now(UTC)
changes_schema = ChangesSchema()


# Current in pu

# Filter to take only branch, connection_type == 2
branch = linedata_distflow.filter(c("connection_type") == 2).select(
    c("line_number").alias("dso_code"),
    c("i_pu").alias("current_limit"),
    c("r_pu"),
    c("x_pu"),
    c("b_pu"),
    pl.lit(base_length).alias("length"),
    pl.lit(BRANCH).alias("concrete_class"),
    pl.lit(default_install_date).alias("start"),
    pl.lit(heartbeat).alias("start_heartbeat"),
    c("line_number").pipe(generate_uuid_col, added_string=BRANCH).alias("uuid"),
    pl.lit(TerminalSide.T1).alias("t1"),
    pl.lit(TerminalSide.T2).alias("t2"),
    ("node_number" + c("node_from").cast(pl.String)).alias("t1_container_fk"),
    ("node_number" + c("node_to").cast(pl.String)).alias("t2_container_fk"),
)
new_tables_pl: dict[str, pl.DataFrame] = {
    "Resource": branch,
    "Equipment": branch,
    "Branch": branch,
}
changes_schema = add_table_to_changes_schema(
    schema=changes_schema, new_tables_pl=new_tables_pl, raw_table_name="branch"
)
changes_schema = generate_connectivity_table(
    changes_schema=changes_schema, eq_table=branch, raw_data_table="branch"
)

## Switch

In [117]:
switch = linedata_distflow.filter(c("connection_type") == 3).select(
    c("line_number").alias("dso_code"),
    pl.lit(SWITCH).alias("concrete_class"),
    pl.lit(default_install_date).alias("start"),
    pl.lit(heartbeat).alias("start_heartbeat"),
    pl.lit(switch_state).alias("normal_open"),
    pl.lit(switch_type).alias("type"),
    pl.lit(switch_command).alias("command"),
    pl.lit(TerminalSide.T1).alias("t1"),
    pl.lit(TerminalSide.T2).alias("t2"),
    ("node_number" + c("node_from").cast(pl.String)).alias("t1_container_fk"),
    ("node_number" + c("node_to").cast(pl.String)).alias("t2_container_fk"),
    c("line_number").pipe(generate_uuid_col, added_string=SWITCH).alias("uuid"),
)
new_tables_pl: dict[str, pl.DataFrame] = {
    "Resource": switch,
    "Equipment": switch,
    "Switch": switch,
}
changes_schema = add_table_to_changes_schema(
    schema=changes_schema, new_tables_pl=new_tables_pl, raw_table_name="switch"
)
changes_schema = generate_connectivity_table(
    changes_schema=changes_schema, eq_table=switch, raw_data_table="switch"
)

In [118]:
changes_schema.connectivity

diff,uuid,start_heartbeat,end_heartbeat,start,end,eq_fk,side,eq_class,abstraction_fk,cn_fk,container_fk,indirect
str,str,"datetime[μs, UTC]","datetime[μs, UTC]","datetime[μs, UTC]","datetime[μs, UTC]",str,str,str,str,str,str,bool
"""+""","""97947fa3-7d23-4093-b824-2cf59c…",2024-12-19 10:36:20.582982 UTC,null,1960-01-01 00:00:00 UTC,null,"""b4fdf239-cfe2-5453-91c3-84c1b2…","""t1""","""branch""","""physical""","""t1""","""node_number1""",false
"""+""","""19371d8b-466f-47c9-baf7-343a1d…",2024-12-19 10:36:20.582982 UTC,null,1960-01-01 00:00:00 UTC,null,"""3edce6fc-e164-5b3e-9b0d-8a1c86…","""t1""","""branch""","""physical""","""t1""","""node_number2""",false
"""+""","""4675243b-885f-4d28-8612-cee24b…",2024-12-19 10:36:20.582982 UTC,null,1960-01-01 00:00:00 UTC,null,"""c173d7f3-f7ca-5f44-98d8-63c1aa…","""t1""","""branch""","""physical""","""t1""","""node_number4""",false
"""+""","""afa0278c-e22e-40e3-9c4e-041f66…",2024-12-19 10:36:20.582982 UTC,null,1960-01-01 00:00:00 UTC,null,"""c33afba9-0379-5d6a-9ca1-743cf7…","""t1""","""branch""","""physical""","""t1""","""node_number5""",false
"""+""","""3da8ba4c-c32e-45b5-b528-c2033f…",2024-12-19 10:36:20.582982 UTC,null,1960-01-01 00:00:00 UTC,null,"""020bd95c-9320-5560-9627-4bbe19…","""t1""","""branch""","""physical""","""t1""","""node_number6""",false
…,…,…,…,…,…,…,…,…,…,…,…,…
"""+""","""3ae17f98-51d4-40ee-8993-ffc65a…",2024-12-19 10:36:20.582982 UTC,null,1960-01-01 00:00:00 UTC,null,"""ec28307e-8860-52db-9611-713cad…","""t2""","""branch""","""physical""","""t2""","""node_number55""",false
"""+""","""7d520d9a-5b75-4529-acb0-7a0369…",2024-12-19 10:36:20.582982 UTC,null,1960-01-01 00:00:00 UTC,null,"""c8a03e72-a228-5007-ad70-3cc3ab…","""t1""","""switch""","""physical""","""t1""",null,false
"""+""","""59f5cc77-e0fc-414f-a8ae-5487cb…",2024-12-19 10:36:20.582982 UTC,null,1960-01-01 00:00:00 UTC,null,"""4f44a759-906d-568d-95b7-a57f77…","""t1""","""switch""","""physical""","""t1""",null,false


In [119]:
# Begin time of the data from matlab (from main_FC.ipynb before)
str(datetime(2020, 4, 4, 23, 00, 0, 0, UTC) - dt.timedelta(hours=192))

'2020-03-27 23:00:00+00:00'

## Parser

In [120]:
# Parse connectivity node
df_topology

node_number,index_branch_1,index_branch_2,index_branch_3,Vnom,Pload,Qload
i64,i64,i64,i64,i64,f64,f64
0,1,2,3,400,null,null
1,4,5,null,400,0.0,0.0
2,6,7,8,400,0.0,0.0
3,9,null,null,400,0.0,0.0
4,null,null,null,400,0.0,0.0
…,…,…,…,…,…,…
53,null,null,null,400,0.0,0.0
54,55,56,null,400,0.000035,-0.000012
55,57,null,null,400,0.0,0.0


In [121]:
def parse_connectivity_node(
    topology_df: pl.DataFrame, changes_schema: ChangesSchema, **kwargs
) -> ChangesSchema:

    cn_voltage_mapping: dict[str, float] = pl_to_dict(
        topology_df.filter(c("KEYWORD") != "TR2")
        .unpivot(
            index=["UN"], on=["t1", "t2"], value_name="cn_fk", variable_name="side"
        )
        .drop_nulls("cn_fk")
        .group_by("cn_fk")
        .agg(c("UN").drop_nulls().first())
        .drop_nulls("UN")[["cn_fk", "UN"]]
    )
    node = topology_df.filter(c("KEYWORD") == "NODE").with_columns(
        (1e3 * c("uuid").replace_strict(cn_voltage_mapping, default=c("UN")))
        .cast(pl.Int32)
        .alias("base_voltage_fk"),  # kV to V
    )

    changes_schema = add_table_to_changes_schema(
        schema=changes_schema,
        new_tables_pl={"ConnectivityNode": node},
        raw_table_name="ConnectivityNode",
    )
    return changes_schema

## Import data to changes schema

### Read grid topology
Put all parts of the network topology (node and equipment) from csv file and put them in dictionary (with equipment names as keys).

In [122]:
# topology : pl.DataFrame = readgridtopology
heartbeat = datetime.now(UTC)
topology

NameError: name 'topology' is not defined

In [ ]:
def sum_downstream_power(col: pl.Expr, df: pl.DataFrame):
    return col.map_elements(
        lambda x: df.filter(c("upstream") == x)["p_line"].sum(), return_dtype=pl.Float64
    )


def calculate_line_power(df: pl.DataFrame):
    return (c("downstream").pipe(sum_downstream_power, df=df) + c("P")) * (1 + c("F"))


def sum_power(df: pl.DataFrame, lv: int):

    return df.with_columns(
        pl.when(c("lv") == lv)
        .then(calculate_line_power(df=df))
        .otherwise(c("p_line"))
        .alias("p_line")
    )


# UP Use for each powerflow
# Down Use only one time
def get_node_level(G: nx.DiGraph) -> dict:
    level_mapping: dict = {}
    for node in reversed(list(nx.topological_sort(G))):
        if not len(list(G.successors(node))):
            level_mapping[node] = 0
        else:
            level_mapping[node] = max(level_mapping[n] for n in G.successors(node)) + 1
    return level_mapping


line_data: pl.DataFrame = pl.DataFrame(
    {
        "downstream": [1, 2, 3, 4, 5, 6, 7, 8],
        "upstream": [None, 1, 2, 1, 4, 4, 4, 6],
        "P": [0, 1, 2, 1, 4, 3, 6, 5],
        "F": [0.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
        "p_line": [0] * 8,
    }
)

grid = nx.DiGraph()

_ = line_data.drop_nulls(subset="upstream").with_columns(
    pl.struct(c("upstream"), c("downstream")).map_elements(
        lambda x: grid.add_edge(x["upstream"], x["downstream"]), return_dtype=pl.Struct
    )
)
level_mapping: dict = get_node_level(G=grid)
line_data = line_data.with_columns(
    c("downstream").replace_strict(level_mapping, default=None).alias("lv")
)

for i in range(line_data["lv"].max() + 1):
    line_data = sum_power(df=line_data, lv=i)

print(line_data.sort("lv"))

shape: (8, 6)
┌────────────┬──────────┬─────┬─────┬────────┬─────┐
│ downstream ┆ upstream ┆ P   ┆ F   ┆ p_line ┆ lv  │
│ ---        ┆ ---      ┆ --- ┆ --- ┆ ---    ┆ --- │
│ i64        ┆ i64      ┆ i64 ┆ f64 ┆ f64    ┆ i64 │
╞════════════╪══════════╪═════╪═════╪════════╪═════╡
│ 3          ┆ 2        ┆ 2   ┆ 0.1 ┆ 2.2    ┆ 0   │
│ 5          ┆ 4        ┆ 4   ┆ 0.1 ┆ 4.4    ┆ 0   │
│ 7          ┆ 4        ┆ 6   ┆ 0.1 ┆ 6.6    ┆ 0   │
│ 8          ┆ 6        ┆ 5   ┆ 0.1 ┆ 5.5    ┆ 0   │
│ 2          ┆ 1        ┆ 1   ┆ 0.1 ┆ 3.52   ┆ 1   │
│ 6          ┆ 4        ┆ 3   ┆ 0.1 ┆ 9.35   ┆ 1   │
│ 4          ┆ 1        ┆ 1   ┆ 0.1 ┆ 23.485 ┆ 2   │
│ 1          ┆ null     ┆ 0   ┆ 0.0 ┆ 27.005 ┆ 3   │
└────────────┴──────────┴─────┴─────┴────────┴─────┘
